<a href="https://colab.research.google.com/github/sraghu678/Titanic-survival-prediction/blob/master/Titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Data analysing and visualising libraray
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#load the data
titanic = sns.load_dataset('titanic')
#read the dataset
titanic.head(5)

In [ ]:
#count the rows and columns
titanic.shape

In [ ]:
#Characterize the information in dataset
titanic.describe()

In [ ]:
#checking the survived people
titanic['survived'].value_counts()

In [ ]:
#draw a plot of survival
sns.countplot(titanic['survived'])

In [ ]:
#visualizing different category of columns
col = ['sex', 'who', 'pclass', 'sibsp',	'parch',	'embarked']
n_rows = 2
n_cols = 3

#the subplot of grid and figure size of each graph
fig, axs = plt.subplots(n_rows, n_cols, figsize = (n_cols * 3.2, n_rows * 3.2) )

for r in range(0, n_rows):
  for c in range(0, n_cols):
    i = r*n_cols + c
    ax = axs[r][c]
    sns.countplot(titanic[col[i]] , hue = titanic['survived'], ax=ax)
    ax.set_title(col[i])
    ax.legend(title ='survived', loc = 'upper right')

plt.tight_layout()



In [ ]:
#draw a bar plot of survival by sex
sns.barplot(x="sex", y="survived", data=titanic)

In [ ]:
#survival rate by sex and class
titanic.pivot_table('survived', index = 'sex', columns= 'pclass')

In [ ]:
#draw a bar plot of survival by sex and class
sns.barplot(x='pclass', y='survived', data=titanic)

In [ ]:
#survival by sex, class, and age
age = pd.cut(titanic['age'], [0,20,80])
titanic.pivot_table('survived', ['sex' , 'pclass', age] )


In [ ]:
#survival based on fare
fare = pd.cut(titanic['fare'], [0,50,100,200,300,400,500,600])
titanic.pivot_table('survived', [fare] )

In [ ]:
#look up all the value get count
for val in titanic:
  print(titanic[val].value_counts())
  print()


**Here alive, deck, embarked_town, who, class have no use**

In [ ]:
#Drop the column
titanic = titanic.drop(['alive', 'deck', 'embark_town', 'who', 'class', 'alone', 'adult_male'], axis = 1)
#removing the missing values
titanic = titanic.dropna(subset= ['embarked', 'age'])

In [ ]:
titanic.shape

In [ ]:
titanic.dtypes

In [ ]:
#print the unique value in columns

print(titanic['sex'].unique())
print(titanic['embarked'].unique())

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

#encode the sex column
titanic.iloc[:, 2] =labelencoder.fit_transform(titanic.iloc[:, 2].values )

#encode the embarked column
titanic.iloc[:, 7] =labelencoder.fit_transform(titanic.iloc[:, 7].values )


In [ ]:
#print the unique value in columns

print(titanic['sex'].unique())
print(titanic['embarked'].unique())

In [ ]:
titanic.dtypes

In [ ]:
#split the data
X = titanic.iloc[:, 1:8].values
Y = titanic.iloc[:, 0].values

In [ ]:
#split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,  test_size = 0.2, random_state = 0)

In [ ]:
#scale the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [ ]:
#testing different models
def models(X_train, Y_train):
  #use logistic regression
  from sklearn.linear_model import LogisticRegression

  log = LogisticRegression(random_state = 0)
  log.fit(X_train, Y_train)

  #use Kneighbors
  from sklearn.neighbors import KNeighborsClassifier

  knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p =2)
  knn.fit(X_train, Y_train)

  # Linear SVC
  from sklearn.svm import LinearSVC

  linear_svc = LinearSVC()
  linear_svc.fit(X_train, Y_train)
  #decision tree classifier
  from sklearn.tree import DecisionTreeClassifier

  tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
  tree.fit(X_train, Y_train)

  # Gaussian Naive Bayes
  from sklearn.naive_bayes import GaussianNB
  from sklearn.metrics import accuracy_score

  gauss = GaussianNB()
  gauss.fit(X_train, Y_train) 

  # Random Forest
  from sklearn.ensemble import RandomForestClassifier

  forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
  forest.fit(X_train, Y_train)
  #print the accuracy for each model
  print(log.score(X_train, Y_train))
  print(knn.score(X_train, Y_train))
  print(linear_svc.score(X_train, Y_train))
  print(tree.score(X_train, Y_train))
  print(gauss.score(X_train, Y_train))
  print(forest.score(X_train, Y_train))

  return log, knn, linear_svc, tree, gauss, forest
   

In [ ]:
#get and train all of the model
model = models(X_train, Y_train)


In [ ]:
#show the confusion matrix and test data accuracy

from sklearn.metrics import confusion_matrix
for i in range( len(model)):
  cm = confusion_matrix(Y_test, model[i].predict(X_test)).ravel()
  #extract TN, FN, TP, FP
  TN, FP, FN, TP = confusion_matrix(Y_test, model[i].predict(X_test)).ravel()
  test_score = (TN+TP)/(TP+TN+FN+FP)
  print(cm)
  print ('Model[{}] Testing Accuracy = "{}"'.format(i, test_score))
  print()

**Here We find highest accuracy of our by Random Forest Classifier which is 83% on our testing dataset**

In [ ]:
#print the prediction of random forest classifier
pred = model[5].predict(X_test)
print(pred)
print()

#print the tested value
print(Y_test)


In [ ]:
#finding survival of user defined value

survival = [[1, 0, 25, 6, 8, 0, 1 ]]

#scaling survial
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
survival_scaled = sc.fit_transform(survival)

#print the survival by random forest classifier
pred = model[5].predict(survival_scaled)
print(pred)

if pred == 0:
  print('you can not survive')
else:
  print('you can survive')


**Hence the conclusion is that female who sit in 1st and 2'nd has more survival chance**